# 승자의 지혜 - 8등 소스코드 분석 

#### train 데이터의 크기가 2GB 정도로 코드를 돌리는 시간이 많이 소요되어 train 데이터 중 100만 개만 사용 (코드 by 정현)
#### ** 파일을 다운받은 경우, 다운받은 'train_ver2_trunc.csv' 파일이 있는 곳으로 경로 위치를 설정해주세요. 파일이 필요한 경우, cell type을 code로 바꾸어 아래에 있는 코드를 실행시켜 주세요 

import pandas as pd

trn = pd.read_csv('C:/Users/jiwon/data/train_ver2.csv') # 다운 받은 파일이 해당 경로에 있는지 확인
trn = trn.sort_values(by=['ncodpers'], axis=0).iloc[1:1000000,]
trn.drop(trn.columns[[0]], axis='columns')

trn.to_csv('C:/Users/jiwon/data/train_ver2_trunc.csv', # 저장할 경로 지정 
           sep=',', na_rep='NaN',
           index=False)

### 0. 데이터 준비 

In [3]:
# 훈련 데이터와 테스트 데이터를 하나의 데이터로 통합하는 코드이다.
def clean_data(fi, fo, header, suffix):
    
    # fi : 훈련/테스트 데이터를 읽어오는 file iterator
    # fo : 통합되는 데이터가 write되는 경로
    # header : 데이터에 header 줄을 추가할 것인지를 결정하는 boolean
    # suffix : 훈련 데이터에는 48개의 변수가 있고, 테스트 데이터에는 24개의 변수만 있다. suffix로 부족한 테스트 데이터 24개분을 공백으로 채운다.

    # csv의 첫줄, 즉 header를 읽어온다
    head = fi.readline().strip("\n").split(",")
    head = [h.strip('"') for h in head]

    # ‘nomprov’ 변수의 위치를 ip에 저장한다
    for i, h in enumerate(head):
        if h == "nomprov":
            ip = i

    # header가 True 일 경우에는, 저장할 파일의 header를 write한다
    if header:
        fo.write("%s\n" % ",".join(head))

    # n은 읽어온 변수의 개수를 의미한다 (훈련 데이터 : 48, 테스트 데이터 : 24)
    n = len(head)
    for line in fi:
        # 파일의 내용을 한줄 씩 읽어와서, 줄바꿈(\n)과 ‘,’으로 분리한다
        fields = line.strip("\n").split(",")

        # ‘nomprov’변수에 ‘,’을 포함하는 데이터가 존재한다. ‘,’으로 분리된 데이터를 다시 조합한다
        if len(fields) > n:
            prov = fields[ip] + fields[ip+1]
            del fields[ip]
            fields[ip] = prov

        # 데이터 개수가 n개와 동일한지 확인하고, 파일에 write한다. 테스트 데이터의 경우, suffix는24개의 공백이다
        assert len(fields) == n
        fields = [field.strip() for field in fields]
        fo.write("%s%s\n" % (",".join(fields), suffix))

In [4]:
# 하나의 데이터로 통합하는 코드를 실행한다. 먼저 훈련 데이터를 write하고, 그 다음으로 테스트 데이터를 write한다. 이제부터 하나의 dataframe만을 다루며 데이터 전처리를 진행한다.
with open("C:/Users/jiwon/data/8th.clean.all.csv", "w") as f:
    clean_data(open("C:/Users/jiwon/data/train_ver2_trunc.csv"), f, True, "") # 경로 확인 
    comma24 = "".join(["," for i in range(24)])
    clean_data(open("C:/Users/jiwon/data/test_ver2.csv"), f, False, comma24)

In [5]:
#### lightgbm 모듈 설치
#### ** 파일이 필요할 경우, cell type을 code로 바꾸어 실행시켜 주세요 

! pip install lightgbm

#### LightGBM

Tree 기반의 러닝 알고리즘을 사용한 gradient boosting framework 입니다. 아래와 같은 장점이 있습니다.

- Faster training speed and higher efficiency. (빠른 훈련 속도와 높은 효율)
- Lower memory usage. (적은 메모리 사용)
- Better accuracy. (높은 정확도)
- Support of parallel and GPU learning. (병렬 처리와 GPU 러닝 지원)
- Capable of handling large-scale data. (큰 scale 데이터 다룰 수 있음)

In [6]:
import math    
import io    

# 파일 압축 용도
import gzip    
import pickle    
import zlib    

# 데이터, 배열을 다루기 위한 기본 라이브러리
import pandas as pd 
import numpy as np

# 범주형 데이터를 수치형으로 변환하기 위한 전처리 도구
from sklearn.preprocessing import LabelEncoder

import sys
sys.path.append('C:\\Users\\jiwon\\data\\kaggle_santander_product_recommendation\\03_Winners_code\\code')
sys.path.append('C:\\Users\\jiwon\\data\\kaggle_santander_product_recommendation\\03_Winners_code\\code\\__pycache__')
import engines
from utils import *

np.random.seed(2016)
transformers = {}

### 1. 데이터 전처리

In [7]:
# “데이터 준비”에서 통합한 데이터를 읽어온다
fname = "C:/Users/jiwon/data/8th.clean.all.csv"
train_df = pd.read_csv(fname) # dtype=dtypes 삭제 

# products는 util.py에서 정의한 24개의 금융 제품이름이다
# 결측값을 0.0으로 대체하고, 정수형으로 변환한다
for prod in products:
    train_df[prod] = train_df[prod].fillna(0.0).astype(np.int8)


C:\Users\jiwon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
train_df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-06-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,...,0,0,0,0,1,1,0,0,0,0
1,2015-12-28,15889,F,ES,V,56.0,1995-01-16,0.0,250.0,1.0,...,0,0,0,0,1,1,0,0,0,0
2,2015-07-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,...,0,0,0,0,1,1,0,0,0,0
3,2015-09-28,15889,F,ES,V,56.0,1995-01-16,0.0,247.0,1.0,...,0,0,0,0,0,1,0,0,0,0
4,2015-01-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,...,0,0,0,0,1,1,0,0,0,0


## apply_transform() 함수에 포함된 4개의 도구함수 설명
### 1. label_encode():  범주형 변수를 수치형으로 변환
데이터프레임 df에서 범주형 변수 name을 LabelEncoder()를 사용하여 수치형으로 변환한다.

사전에 정의한 dict()인 transforms에 label encoding을 수행한 변수명을 기록하여, 데이터프레임 df에 동일한 변수를 label encoding 할 때에는 기존의 LabelEncoder()를 재활용한다.

In [9]:
def label_encode(df, features, name):
    
    df[name] = df[name].astype('str') # 데이터 프레임 df의 변수 name의 값을 모두 string으로 변환한다
                                      # 이미 label_encode 했던 변수일 경우, transformer[name]에 있는 LabelEncoder()를 재활용한다
    if name in transformers:
        df[name] = transformers[name].transform(df[name])
    # 처음 보는 변수일 경우, transformer에 LabelEncoder()를 저장하고, .fit_transform() 함수로 label encoding을 수행한다
    else: # train
        transformers[name] = LabelEncoder()
        df[name] = transformers[name].fit_transform(df[name])
    # label encoding한 변수는 features 리스트에 추가한다
    features.append(name)
    

### 2. encode_top(): 빈도가 높은 데이터에 대한 정보 추출
encode_top(s, count=100, dtype=np.int8) 함수는 pd.Series에서 빈도가 가장 높은 100개의 고유값을 순위로 대체하고 

그 외 빈도가 낮은 값은 모두 0으로 변환한 새로운 pd.Series를 반환한다.

데이터 전체가 아닌 고빈도 데이터에 대한 정보를 추출하는 함수이다.

In [10]:
def encode_top(s, count=100, dtype=np.int8):

    uniqs, freqs = np.unique(s, return_counts=True) # 모든 고유값에 대한 빈도를 계산한다
    
    top = sorted(zip(uniqs,freqs), key=lambda vk: vk[1], reverse = True)[:count] # 빈도 Top 100을 추출한다
    
    top_map = {uf[0]: l+1 for uf, l in zip(top, range(len(top)))} # { 기존 데이터 : 순위 } 를 나타내는 dict()를 생성한다
    # 고빈도 100개의 데이터는 순위로 대체하고, 그 외는 0으로 대체한다
    return s.map(lambda x: top_map.get(x, 0)).astype(dtype) #top_map에서 x에 해당하는 key값을 가져오고 없을 경우 0을 넣는다


### 3. date_to_float(), date_to_int(): 날짜 데이터를 숫자로 변환
date_to_float(str_date) 함수는 입력으로 들어오는 날짜 데이터를 숫자로 변환하는 함수이다. 

입력값이 결측값일 경우에는 np.nan 반환하지만, 입력값이 문자열 형태의 날짜 데이터일 경우 ("2015-06-28"), '년도*12 + 월' 이라는 계산으로 날짜 데이터를 소수로 환산하여 반환한다.

날짜 데이터를 단순 월 단위 수치형 데이터로 변환한 것이다.

date_to_int(str_date)는 월 단위 수치형으로 변환된 데이터를 1 ~ 18 사이의 값으로 변환한다.

In [11]:
# 날짜 데이터를 월 단위 숫자로 변환하는 함수
def date_to_float(str_date):
    if str_date.__class__ is float and math.isnan(str_date) or str_date == "":
        return np.nan
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    float_date = float(Y) * 12 + float(M)
    return float_date

# 날짜 데이터를 월 단위 숫자로 변환하되 1~18 사이로 제한하는 함수
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] # "2016-05-28"
    int_date = (int(Y) - 2015) * 12 + int(M)
    assert 1 <= int_date <= 12 + 6
    return int_date


### 4. custom_one_hot(): 범주형 변수 안에 존재하는 고유값을 새로운 이진 변수로 생성
custom_on_hot(df,features,name,names) 함수는 범주형 변수를 입력받아 변수 안에 존재하는 고유값을 새로운 이진 변수로 생성하는 one-hot-encoding을 수행한다.

범주형 데이터를 하나의 열에서 label encoding 하는 것보다 표현력이 높아지지만, 고유값의 숫자만큼 데이터의 열이 늘어나기 때문에

고유값이 적은 데이터에서 선호되는 피처엔지니어링 기법이다.

sklearn,preprocessing.OneHotEncoding, pandas.get_dummies 함수는 one-hot-encoding 기능을 지원하지만, 8등 코드에서는 직접 one-hot-encoder를 구축했다.

In [12]:
def custom_one_hot(df, features, name, names, dtype=np.int8, check=False):
    for n, val in names.items():
        
        new_name = "%s_%s" % (name, n) # 신규 변수명을 “변수명_숫자”로 지정한다
        
        df[new_name] = df[name].map(lambda x: 1 if x == val else 0).astype(dtype) # 기존 변수에서 해당 고유값을 가지면 1, 그 외는 0인 이진 변수를 생성한다
        features.append(new_name)

### apply_transform(): 앞에서 설명한 4개의 도구 함수를 사용하여 총 48개의 변수에 대한 데이터 전처리와 피처 엔지니어링을 수행

#### 주된 데이터 전처리와 피처엔지니어링 내용은 다음과 같다.


- 결측값 대체: 데이터 내에 존재하는 결측값을 0.0 혹은 1.0 등으로 대체한다


- 범주형 데이터 label encoding: 범주형으로 표현되는 데이터를 sklearn.preprocessing의 LabelEncoder 도구를 사용해 수치형으로 변환한다.


- 고빈도 Top 100개를 빈도 순위로 변환: 특정 변수에서 빈도가 높은 값을 순위로 변환하여, 고빈도 데이터에 대한 선형 관계를 추출한다.


- 수치형 변수 log transformation: log transformation은 데이터 내의 대소관계를 유지하면서 포함된 값들의 차이를 줄여주는 역할을 한다.


- 날짜 데이터에서 년/월을 추출: "2015-06-28"과 같은 문자열 데이터에서 연도와 월을 추출한다.


- 날짜 데이터 간의 차이값으로 파생 변수 생성: 2개 날짜 데이터의 차이값을 통하여 상대적인 거리 변수를 생성한다.


- one-hot-encoding 변수 생성: 범주형 데이터의 표현력을 높이기 위하여, 모든 고유값을 새로운 이진 변수로 생성한다.

In [13]:
def apply_transforms(train_df):
    # 학습에 사용할 변수를 저장할 features 리스트를 생성한다
    features = []

    # 두 변수를 label_encode() 한다 (범주형 -> 수치형)
    label_encode(train_df, features, "canal_entrada") #고객 유입 채널
    label_encode(train_df, features, "pais_residencia") #고객 거주 국가

    # age의 결측값을 0.0으로 대체하고, 모든 값을 정수로 변환한다.
    train_df["age"] = train_df["age"].fillna(0.0).astype(np.int16)
    features.append("age")

    # renta(가구 총수입)의 결측값을 1.0으로 대체하고, log를 씌워 분포를 변형한다
    train_df["renta"].fillna(1.0, inplace=True)
    train_df["renta"] = train_df["renta"].map(math.log)
    features.append("renta")

    # encode_top() 함수를 이용하여 고빈도 100개의 순위를 추출한다
    train_df["renta_top"] = encode_top(train_df["renta"])  
    features.append("renta_top")

    # 결측값 혹은 음수를 0으로 대체하고, 나머지 값은 +1.0 은 한 후에, 정수로 변환한다
    #antiguedad: 은행 거래 누적 기간 (월)
    train_df["antiguedad"] = train_df["antiguedad"].map(lambda x: 0.0 if x < 0 or math.isnan(x) else x+1.0).astype(np.int16)
    features.append("antiguedad")

    # 결측값을 0.0으로 대체하고, 정수로 변환한다
    train_df["tipodom"] = train_df["tipodom"].fillna(0.0).astype(np.int8) #주소 유형
    features.append("tipodom")
    train_df["cod_prov"] = train_df["cod_prov"].fillna(0.0).astype(np.int8) #지방 코드 (주소 기반)
    features.append("cod_prov")

    # fecha_dato(날짜)에서 월/년도를 추출하여 정수값으로 변환한다
    train_df["fecha_dato_month"] = train_df["fecha_dato"].map(lambda x: int(x.split("-")[1])).astype(np.int8)
    features.append("fecha_dato_month")
    train_df["fecha_dato_year"] = train_df["fecha_dato"].map(lambda x: float(x.split("-")[0])).astype(np.int16)
    features.append("fecha_dato_year")

    # 결측값을 0.0으로 대체하고, fecha_alta(고객이 은행과 첫 계약을 한 날짜)에서 월/년도를 추출하여 정수값으로 변환한다
    # x.__class__는 결측값일 경우 float를 반환하기 때문에, 결측값 탐지용으로 사용하고 있다
    train_df["fecha_alta_month"] = train_df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[1])).astype(np.int8)
    features.append("fecha_alta_month")
    train_df["fecha_alta_year"] = train_df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[0])).astype(np.int16)
    features.append("fecha_alta_year")

    # date_to_float() 함수 이용하여 날짜 데이터를 월 기준 수치형 변수로 변환한다
    train_df["fecha_dato_float"] = train_df["fecha_dato"].map(date_to_float)
    train_df["fecha_alta_float"] = train_df["fecha_alta"].map(date_to_float)

    # fecha_dato 와 fecha_alto의 월 기준 수치형 변수의 차이값을 파생 변수로 생성한다
    train_df["dato_minus_alta"] = train_df["fecha_dato_float"] - train_df["fecha_alta_float"] #(현재 날짜) - (첫계약 날짜)
    features.append("dato_minus_alta")

    # date_to_int() 함수 이용하여 날짜 데이터를 월 기준 수치형 변수로 변환한다 (1 ~ 18 사이 값으로 제한)
    train_df["int_date"] = train_df["fecha_dato"].map(date_to_int).astype(np.int8)

    # custom_one_hot() 함수 이용하여 자체 개발한 one-hot-encoding을 수행한다 (범주형 변수 -> 이진 변수)
    custom_one_hot(train_df, features, "indresi", {"n":"N"}) #거주 지표
    custom_one_hot(train_df, features, "indext", {"s":"S"}) #외국인 지표
    custom_one_hot(train_df, features, "conyuemp", {"n":"N"}) #배우자 지표
    custom_one_hot(train_df, features, "sexo", {"h":"H", "v":"V"}) #성별
    custom_one_hot(train_df, features, "ind_empleado", {"a":"A", "b":"B", "f":"F", "n":"N"}) #고용 지표
    custom_one_hot(train_df, features, "ind_nuevo", {"new":1}) #신규 고객 지표 
    custom_one_hot(train_df, features, "segmento", {"top":"01 - TOP", "particulares":"02 - PARTICULARES", "universitario":"03 - UNIVERSITARIO"}) #분류(VIP / 개인 / 대졸)
    custom_one_hot(train_df, features, "indfall", {"s":"S"}) #고객 사망 여부
    custom_one_hot(train_df, features, "tiprel_1mes", {"a":"A", "i":"I", "p":"P", "r":"R"}, check=True) #월초 기준 고객 관계 유형
    custom_one_hot(train_df, features, "indrel", {"1":1, "99":99}) #고객 등급

    # 결측값을 0.0으로 대체하고, 그 외는 +1.0을 더하고, 정수로 변환한다
    # ind_actividad_cliente: 활발성 지표
    train_df["ind_actividad_cliente"] = train_df["ind_actividad_cliente"].map(lambda x: 0.0 if math.isnan(x) else x+1.0).astype(np.int8)
    features.append("ind_actividad_cliente")

    # 결측값을 0.0으로 대체하고, “P”를 5로 대체하고, 정수로 변환한다
    # indrel_1mes: 월초 기준 고객 등급
    train_df["indrel_1mes"] = train_df["indrel_1mes"].map(lambda x: 5.0 if x == "P" else x).astype(float).fillna(0.0).astype(np.int8)
    features.append("indrel_1mes")
    
    # 데이터 전처리/피쳐 엔지니어링이 1차적으로 완료된 데이터 프레임 train_df와 학습에 사용할 변수 리스트 features를 tuple 형태로 반환한다
    return train_df, tuple(features)


In [14]:
# 48개의 변수마다 전처리/피처 엔지니어링을 적용한다
train_df, features = apply_transforms(train_df)

In [35]:
train_df.head() #열 개수가 77개로 늘어남

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,segmento_top,segmento_particulares,segmento_universitario,indfall_s,tiprel_1mes_a,tiprel_1mes_i,tiprel_1mes_p,tiprel_1mes_r,indrel_1,indrel_99
0,2015-06-28,15889,F,36,V,56,1995-01-16,0.0,246,1.0,...,1,0,0,0,1,0,0,0,1,0
1,2015-12-28,15889,F,36,V,56,1995-01-16,0.0,251,1.0,...,1,0,0,0,1,0,0,0,1,0
2,2015-07-28,15889,F,36,V,56,1995-01-16,0.0,246,1.0,...,1,0,0,0,1,0,0,0,1,0
3,2015-09-28,15889,F,36,V,56,1995-01-16,0.0,248,1.0,...,1,0,0,0,1,0,0,0,1,0
4,2015-01-28,15889,F,36,V,56,1995-01-16,0.0,246,1.0,...,1,0,0,0,1,0,0,0,1,0


In [40]:
print(features) #학습에 사용할 변수들
len(features)

('canal_entrada', 'pais_residencia', 'age', 'renta', 'renta_top', 'antiguedad', 'tipodom', 'cod_prov', 'fecha_dato_month', 'fecha_dato_year', 'fecha_alta_month', 'fecha_alta_year', 'dato_minus_alta', 'indresi_n', 'indext_s', 'conyuemp_n', 'sexo_h', 'sexo_v', 'ind_empleado_a', 'ind_empleado_b', 'ind_empleado_f', 'ind_empleado_n', 'ind_nuevo_new', 'segmento_top', 'segmento_particulares', 'segmento_universitario', 'indfall_s', 'tiprel_1mes_a', 'tiprel_1mes_i', 'tiprel_1mes_p', 'tiprel_1mes_r', 'indrel_1', 'indrel_99', 'ind_actividad_cliente', 'indrel_1mes')


35

## 금융 변수의 lag 데이터 생성
시계열 문제에서 많이 사용되는 파생 변수의 하나로써, 해당 변수의 n 단위 이전의 값을 lag-n 데이터라고 한다.


시계열 경진대회에서는 lag-n 데이터가 유의미한 성능 개선을 보이는 경우가 종종 있다.

## lag 변수를 생성하기 위한 2개의 도구 함수


### 1. make_prev_df() 함수: 금융 변수에 대한 lag 데이터를 직접 생성하는 함수
make_prev_df(train_df,step) 함수: 24개의 금융 변수에 대한 lag 데이터를 직접 생성하는 함수이다. 

apply_transform() 함수에서 "fecha_dato" 날짜 데이터를 1~18 사이의 정수로 변환한 int_date 변수를 사용하여 각 24개의 금융 변수의 값을 step 개월 만큼 이동시켜 lag 변수를 생성한다.

In [18]:
def make_prev_df(train_df, step):
    # 새로운 데이터 프레임에 ncodpers(고객식별번호)를 추가하고, int_date를 step만큼 이동시킨 값을 넣는다
    prev_df = pd.DataFrame()
    prev_df["ncodpers"] = train_df["ncodpers"]
    prev_df["int_date"] = train_df["int_date"].map(lambda x: x+step).astype(np.int8)

    # 변수명_prev1” 형태의 lag 변수를 생성한다
    prod_features = ["%s_prev%s" % (prod, step) for prod in products] #lag 변수
    for prod, prev in zip(products, prod_features):
        prev_df[prev] = train_df[prod]

    return prev_df, tuple(prod_features)


prev_df = pd.DataFrame() #lag 데이터
prev_df["ncodpers"] = train_df["ncodpers"] ; prev_df.head()
prev_df["int_date"] = train_df["int_date"].map(lambda x: x+1).astype(np.int8) 
prev_df.head() #예를 들어, step=1인 경우 (lag-1)

train_df["int_date"].head()

prod_features = ["%s_prev%s" % (prod, 1) for prod in products] #lag 변수
print(prod_features)

for prod, prev in zip(products, prod_features): #그냥 상품변수와 lag된 상품변수에 대해서
        prev_df[prev] = train_df[prod] #prev_df의 lag상품변수 column에 원래 데이터 상품을 넣는다

### 2. join_with_prev(): 기존 데이터에 lag 데이터 조인
join_with_prev(df, prev_df, how)함수는 기존의 train_df에 lag 데이터를 조인하는 함수이다.

In [19]:
def join_with_prev(df, prev_df, how):
    # pandas merge 함수를 통해 join
    df = df.merge(prev_df, on=["ncodpers", "int_date"], how=how) #고객식별번호와 1~18로 변환된 날짜 기준으로 merge
    # 24개 금융 변수를 소수형으로 변환한다
    for f in set(prev_df.columns.values.tolist()) - set(["ncodpers", "int_date"]):
        df[f] = df[f].astype(np.float16)
    return df

1차적으로 전처리 및 피처 엔지니어링이 완료된 train_df를 기준으로 최대 lag-5 변수를 생성하고 train_df에 조인한다.

이번 경진대회에서 가장 중요한 역할을 한 변수는 lag-5 변수였으므로 이 과정은 최종 성능에 큰 영향을 준다.

In [20]:
prev_dfs = []
prod_features = None

use_features = frozenset([1,2]) #frozenset: 한번 만들면 변경 불가능

# 1 ~ 5까지의 step에 대하여 make_prev_df()를 통해 lag-n 데이터를 생성한다
for step in range(1,6):
    prev1_train_df, prod1_features = make_prev_df(train_df, step) #생성된 lag데이터, lag변수
    
    # 생성한 lag 데이터는 prev_dfs 리스트에 저장한다
    prev_dfs.append(prev1_train_df)
    
    # features에는 lag-1,2만 추가한다
    if step in use_features:
        features += prod1_features
    # prod_features에는 lag-1의 변수명만 저장한다
    if step == 1:
        prod_features = prod1_features

In [21]:
prev_dfs #생성된 lag1~lag5 데이터

[         ncodpers  int_date  ind_ahor_fin_ult1_prev1  ind_aval_fin_ult1_prev1  \
 0           15889         7                        0                        0   
 1           15889        13                        0                        0   
 2           15889         8                        0                        0   
 3           15889        10                        0                        0   
 4           15889         2                        0                        0   
 ...           ...       ...                      ...                      ...   
 1929609    660237        19                        0                        0   
 1929610    660238        19                        0                        0   
 1929611    660240        19                        0                        0   
 1929612    660243        19                        0                        0   
 1929613    660248        19                        0                        0   
 
          ind_

생성한 lag-5 변수를 통합 데이터인 train_df에 추가한다.
lag-1 변수는 inner join으로 추가하고, 그 외는 left-join으로 데이터에 추가한다.
join_with_prev()를 보면 두 데이터 프레임을 키 값 'ncodepers', 'int_date' 를 기준으로 통합한다.
inner join은 해당 키 값이 두 데이터 프레임에 모두 존재하는 데이터를 병합하고, 그 외 데이터는 제거한다.
lag-1 변수가 통합된 이후에는 train_df의 키 값만을 기준으로 통합하는 left join을 수행하여 lag 데이터를 추가해간다. 

In [22]:
for i, prev_df in enumerate(prev_dfs):
    how = "inner" if i == 0 else "left" #lag-1 변수는 inner join으로 추가하고, 그 외는 left-join으로 데이터에 추가
    train_df = join_with_prev(train_df, prev_df, how=how)

In [26]:
train_df.head()

,ncodpers,int_date,fecha_dato,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,...,ind_hip_fin_ult1_prev2,ind_plan_fin_ult1_prev2,ind_pres_fin_ult1_prev2,ind_reca_fin_ult1_prev2,ind_tjcr_fin_ult1_prev2,ind_valo_fin_ult1_prev2,ind_viv_fin_ult1_prev2,ind_nomina_ult1_prev2,ind_nom_pens_ult1_prev2,ind_recibo_ult1_prev2
0,15889,6,2015-06-28,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,15889,12,2015-12-28,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,15889,7,2015-07-28,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,15889,9,2015-09-28,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,15889,14,2016-02-28,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [23]:
###AttributeError: 'DataFrame' object has no attribute 'as_matrix'

# 24개의 금융 변수에 대해서 for loop을 돈다
for prod in products:
    # [1~3], [1~5], [2~5] 의 3개 구간에 대해서 표준편차를 구한다
    for begin, end in [(1,3),(1,5),(2,5)]:
        prods = ["%s_prev%s" % (prod, i) for i in range(begin,end+1)]
        mp_df = train_df.as_matrix(columns=prods)
        stdf = "%s_std_%s_%s" % (prod,begin,end)

        # np.nanstd로 표준편차를 구하고, features에 신규 파생 변수 이름을 추가한다
        train_df[stdf] = np.nanstd(mp_df, axis=1)
        features += (stdf,)

    # [2~3], [2~5] 의 2개 구간에 대해서 최소값/최대값을 구한다
    for begin, end in [(2,3),(2,5)]:
        prods = ["%s_prev%s" % (prod, i) for i in range(begin,end+1)]
        mp_df = train_df.as_matrix(columns=prods)

        minf = "%s_min_%s_%s"%(prod,begin,end)
        train_df[minf] = np.nanmin(mp_df, axis=1).astype(np.int8)

        maxf = "%s_max_%s_%s"%(prod,begin,end)
        train_df[maxf] = np.nanmax(mp_df, axis=1).astype(np.int8)

        features += (minf,maxf,)

AttributeError: 'DataFrame' object has no attribute 'as_matrix'

In [27]:
# 고객 고유 식별 번호(ncodpers), 정수로 표현한 날짜(int_date), 실제 날짜(fecha_dato), 24개의 금융 변수(products)와 학습에 사용하기 위해 전처리/피쳐 엔지니어링한 변수(features)가 주요 변수이다.
leave_columns = ["ncodpers", "int_date", "fecha_dato"] + list(products) + list(features)
# 중복값이 없는지 확인한다
assert len(leave_columns) == len(set(leave_columns)) #assert 조건: 조건이 맞지 않을 경우 에러를 발생시킨다.
# train_df에서 주요 변수만을 추출한다
train_df = train_df[leave_columns]

In [28]:
train_df, features, prod_features

(        ncodpers  int_date  fecha_dato  ind_ahor_fin_ult1  ind_aval_fin_ult1  \
 0          15889         6  2015-06-28                  0                  0   
 1          15889        12  2015-12-28                  0                  0   
 2          15889         7  2015-07-28                  0                  0   
 3          15889         9  2015-09-28                  0                  0   
 4          15889        14  2016-02-28                  0                  0   
 ...          ...       ...         ...                ...                ...   
 998470    140745        18  2016-06-28                  0                  0   
 998471    140747        18  2016-06-28                  0                  0   
 998472    140748        18  2016-06-28                  0                  0   
 998473    140757        18  2016-06-28                  0                  0   
 998474    131627        18  2016-06-28                  0                  0   
 
         ind_cco_fin_ult1 

### 3. 머신러닝 모델

In [ ]:
train_predict(all_df, features, prod_features, "2016-05-28", cv=True)
train_predict(all_df, features, prod_features, "2016-06-28", cv=False)

In [30]:
def train_predict(all_df, features, prod_features, str_date, cv):
    # all_df : 통합 데이터
    # features : 학습에 사용할 변수
    # prod_features : 24개 금융 변수
    # str_date : 예측 결과물을 산출하는 날짜. 2016-05-28일 경우, 훈련 데이터의 일부이며 정답을 알고 있기에 교차 검증을 의미하고, 2016-06-28일 경우, 캐글에 업로드하기 위한 테스트 데이터 예측 결과물을 생성한다
    # cv : 교차 검증 실행 여부

    # str_date로 예측 결과물을 산출하는 날짜를 지정한다
    test_date = date_to_int(str_date)
    # 훈련 데이터는 test_date 이전의 모든 데이터를 사용한다
    train_df = all_df[all_df.int_date < test_date]
    # 테스트 데이터를 통합 데이터에서 분리한다
    test_df = pd.DataFrame(all_df[all_df.int_date == test_date])

    # 신규 구매 고객만을 훈련 데이터로 추출한다
    X = []
    Y = []
    for i, prod in enumerate(products):
        prev = prod + "_prev1"
        # 신규 구매 고객을 prX에 저장한다
        prX = train_df[(train_df[prod] == 1) & (train_df[prev] == 0)]
        # prY에는 신규 구매에 대한 label 값을 저장한다
        prY = np.zeros(prX.shape[0], dtype=np.int8) + i
        X.append(prX)
        Y.append(prY)

    XY = pd.concat(X)
    Y = np.hstack(Y)
    # XY는 신규 구매 데이터만 포함한다
    XY["y"] = Y

    # 메모리에서 변수 삭제
    del train_df
    del all_df

    # 데이터별 가중치를 계산하기 위해서 새로운 변수 (ncodpers + fecha_dato)를 생성한다
    XY["ncodepers_fecha_dato"] = XY["ncodpers"].astype(str) + XY["fecha_dato"]
    uniqs, counts = np.unique(XY["ncodepers_fecha_dato"], return_counts=True)
    # 자연 상수(e)를 통해서, count가 높은 데이터에 낮은 가중치를 준다
    weights = np.exp(1/counts - 1)

    # 가중치를 XY 데이터에 추가한다
    wdf = pd.DataFrame()
    wdf["ncodepers_fecha_dato"] = uniqs
    wdf["counts"] = counts
    wdf["weight"] = weights
    XY = XY.merge(wdf, on="ncodepers_fecha_dato")

    # 교차 검증을 위하여 XY를 훈련:검증 (8:2)로 분리한다
    mask = np.random.rand(len(XY)) < 0.8
    XY_train = XY[mask]
    XY_validate = XY[~mask]

    # 테스트 데이터에서 가중치는 모두 1이다
    test_df["weight"] = np.ones(len(test_df), dtype=np.int8)

    # 테스트 데이터에서 “신규 구매” 정답값을 추출한다. 
    test_df["y"] = test_df["ncodpers"]
    Y_prev = test_df.as_matrix(columns=prod_features)
    for prod in products:
        prev = prod + "_prev1"
        padd = prod + "_add"
        # 신규 구매 여부를 구한다
        test_df[padd] = test_df[prod] - test_df[prev]

    test_add_mat = test_df.as_matrix(columns=[prod + "_add" for prod in products])
    C = test_df.as_matrix(columns=["ncodpers"])
    test_add_list = [list() for i in range(len(C))]
    # 평가 척도 MAP@7 계산을 위하여, 고객별 신규 구매 정답값을 test_add_list에 기록한다
    count = 0
    for c in range(len(C)):
        for p in range(len(products)):
            if test_add_mat[c,p] > 0:
                test_add_list[c].append(p)
                count += 1
    
    # 교차 검증에서, 테스트 데이터로 분리된 데이터가 얻을 수 있는 최대 MAP@7 값을 계산한다. 
    if cv:
        max_map7 = mapk(test_add_list, test_add_list, 7, 0.0)
        map7coef = float(len(test_add_list)) / float(sum([int(bool(a)) for a in test_add_list]))
        print("Max MAP@7", str_date, max_map7, max_map7*map7coef)

    # LightGBM 모델 학습 후, 예측 결과물을 저장한다
    Y_test_lgbm = engines.lightgbm(XY_train, XY_validate, test_df, features, XY_all = XY, restore = (str_date == "2016-06-28"))
    test_add_list_lightgbm = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.lightgbm.csv.gz" % str_date, "wb"), Y_test_lgbm - Y_prev, C)

    # 교차 검증일 경우, LightGBM 모델의 테스트 데이터 MAP@7 평가 척도를 출력한다
    if cv:
        map7lightgbm = mapk(test_add_list, test_add_list_lightgbm, 7, 0.0)
        print("LightGBMlib MAP@7", str_date, map7lightgbm, map7lightgbm*map7coef)

    # XGBoost 모델 학습 후, 예측 결과물을 저장한다
    Y_test_xgb = engines.xgboost(XY_train, XY_validate, test_df, features, XY_all = XY, restore = (str_date == "2016-06-28"))
    test_add_list_xgboost = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.xgboost.csv.gz" % str_date, "wb"), Y_test_xgb - Y_prev, C)

    # 교차 검증일 경우, XGBoost 모델의 테스트 데이터 MAP@7 평가 척도를 출력한다
    if cv:
        map7xgboost = mapk(test_add_list, test_add_list_xgboost, 7, 0.0)
        print("XGBoost MAP@7", str_date, map7xgboost, map7xgboost*map7coef)

    # 곱셈 후, 제곱근을 구하는 방식으로 앙상블을 수행한다
    Y_test = np.sqrt(np.multiply(Y_test_xgb, Y_test_lgbm))
    # 앙상블 결과물을 저장하고, 테스트 데이터에 대한 MAP@7 를 출력한다
    test_add_list_xl = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.xgboost-lightgbm.csv.gz" % str_date, "wb"), Y_test - Y_prev, C)

    # 정답값인 test_add_list와 앙상블 모델의 예측값을 mapk 함수에 넣어, 평가 척도 점수를 확인한다
    if cv:
        map7xl = mapk(test_add_list, test_add_list_xl, 7, 0.0)
        print("XGBoost+LightGBM MAP@7", str_date, map7xl, map7xl*map7coef)

In [31]:
import os

# xgboost, lightgbm 라이브러리
import xgboost as xgb
import lightgbm as lgbm

from utils import *

In [32]:
# XGBoost 모델을 학습하는 함수이다
def xgboost(XY_train, XY_validate, test_df, features, XY_all=None, restore=False):
    # 최적의 parameter를 지정한다
    param = {
        'objective': 'multi:softprob',
        'eta': 0.1,
        'min_child_weight': 10,
        'max_depth': 8,
        'silent': 1,
        # 'nthread': 16,
        'eval_metric': 'mlogloss',
        'colsample_bytree': 0.8,
        'colsample_bylevel': 0.9,
        'num_class': len(products),
    }

    if not restore:
        # 훈련 데이터에서 X, Y, weight를 추출한다. as_matrix를 통해 메모리 효율적으로 array만 저장한다
        X_train = XY_train.as_matrix(columns=features)
        Y_train = XY_train.as_matrix(columns=["y"])
        W_train = XY_train.as_matrix(columns=["weight"])
        # xgboost 전용 데이터형식으로 변환한다
        train = xgb.DMatrix(X_train, label=Y_train, feature_names=features, weight=W_train)

        # 검증 데이터에 대해서 동일한 작업을 진행한다
        X_validate = XY_validate.as_matrix(columns=features)
        Y_validate = XY_validate.as_matrix(columns=["y"])
        W_validate = XY_validate.as_matrix(columns=["weight"])
        validate = xgb.DMatrix(X_validate, label=Y_validate, feature_names=features, weight=W_validate)

        # XGBoost 모델을 학습한다. early_stop 조건은 20번이며, 최대 1000개의 트리를 학습한다
        evallist  = [(train,'train'), (validate,'eval')]
        model = xgb.train(param, train, 1000, evals=evallist, early_stopping_rounds=20)
        # 학습된 모델을 저장한다
        pickle.dump(model, open("next_multi.pickle", "wb"))
    else:
        # “2016-06-28” 테스트 데이터를 사용할 시에는, 사전에 학습된 모델을 불러온다
        model = pickle.load(open("next_multi.pickle", "rb"))
    # 교차 검증으로 최적의 트리 개수를 정한다
    best_ntree_limit = model.best_ntree_limit

    if XY_all is not None:
        # 전체 훈련 데이터에 대해서 X, Y, weight 를 추출하고, XGBoost 전용 데이터 형태로 변환한다
        X_all = XY_all.as_matrix(columns=features)
        Y_all = XY_all.as_matrix(columns=["y"])
        W_all = XY_all.as_matrix(columns=["weight"])
        all_data = xgb.DMatrix(X_all, label=Y_all, feature_names=features, weight=W_all)

        evallist  = [(all_data,'all_data')]
        # 학습할 트리 개수를 전체 훈련 데이터가 늘어난 만큼 조정한다
        best_ntree_limit = int(best_ntree_limit * (len(XY_train) + len(XY_validate)) / len(XY_train))
        # 모델 학습!
        model = xgb.train(param, all_data, best_ntree_limit, evals=evallist)

    # 변수 중요도를 출력한다. 학습된 XGBoost 모델에서 .get_fscore()를 통해 변수 중요도를 확인할 수 있다
    print("Feature importance:")
    for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
        print(kv)

    # 예측에 사용할 테스트 데이터를 XGBoost 전용 데이터로 변환한다. 이 때, weight는 모두 1이기에, 별도로 작업하지 않는다
    X_test = test_df.as_matrix(columns=features)
    test = xgb.DMatrix(X_test, feature_names=features)

    # 학습된 모델을 기반으로, best_ntree_limit개의 트리를 기반으로 예측한다
    return model.predict(test, ntree_limit=best_ntree_limit)


In [33]:
def lightgbm(XY_train, XY_validate, test_df, features, XY_all=None, restore=False):
    # 훈련 데이터, 검증 데이터 X, Y, weight 추출 후, LightGBM 전용 데이터로 변환한다
    train = lgbm.Dataset(XY_train[list(features)], label=XY_train["y"], weight=XY_train["weight"], feature_name=features)
    validate = lgbm.Dataset(XY_validate[list(features)], label=XY_validate["y"], weight=XY_validate["weight"], feature_name=features, reference=train)

    # 다양한 실험을 통해 얻은 최적의 학습 parameter
    params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'multiclass',
        'num_class': 24,
        'metric' : {'multi_logloss'},
        'is_training_metric': True,
        'max_bin': 255,
        'num_leaves' : 64,
        'learning_rate' : 0.1,
        'feature_fraction' : 0.8,
        'min_data_in_leaf': 10,
        'min_sum_hessian_in_leaf': 5,
        # 'num_threads': 16,
    }

    if not restore:
        # XGBoost와 동일하게 훈련/검증 데이터를 기반으로 최적의 트리 개수를 계산한다
        model = lgbm.train(params, train, num_boost_round=1000, valid_sets=validate, early_stopping_rounds=20)
        best_iteration = model.best_iteration
        # 학습된 모델과 최적의 트리 개수 정보를 저장한다
        model.save_model("tmp/lgbm.model.txt")
        pickle.dump(best_iteration, open("tmp/lgbm.model.meta", "wb"))
    else:
        model = lgbm.Booster(model_file="tmp/lgbm.model.txt")
        best_iteration = pickle.load(open("tmp/lgbm.model.meta", "rb"))

    if XY_all is not None:
        # 전체 훈련 데이터에는 늘어난 양만큼 트리 개수를 늘린다
        best_iteration = int(best_iteration * len(XY_all) / len(XY_train))
        # 전체 훈련 데이터에 대한 LightGBM 전용 데이터를 생성한다
        all_train = lgbm.Dataset(XY_all[list(features)], label=XY_all["y"], weight=XY_all["weight"], feature_name=features)
        # LightGBM 모델 학습!
        model = lgbm.train(params, all_train, num_boost_round=best_iteration)
        model.save_model("tmp/lgbm.all.model.txt")

    # LightGBM 모델이 제공하는 변수 중요도 기능을 통해 변수 중요도를 출력한다
    print("Feature importance by split:")
    for kv in sorted([(k,v) for k,v in zip(features, model.feature_importance("split"))], key=lambda kv: kv[1], reverse=True):
        print(kv)
    print("Feature importance by gain:")
    for kv in sorted([(k,v) for k,v in zip(features, model.feature_importance("gain"))], key=lambda kv: kv[1], reverse=True):
        print(kv)

    # 테스트 데이터에 대한 예측 결과물을 return한다
    return model.predict(test_df[list(features)], num_iteration=best_iteration)


In [35]:
def make_submission(f, Y_test, C):
    Y_ret = []
    # 파일의 첫 줄에 header를 쓴다
    f.write("ncodpers,added_products\n".encode('utf-8'))
    # 고객 식별 번호(C)와, 예측 결과물(Y_test)의 for loop
    for c, y_test in zip(C, Y_test):
        # (확률값, 금융 변수명, 금융 변수 id)의 tuple을 구한다
        y_prods = [(y,p,ip) for y,p,ip in zip(y_test, products, range(len(products)))]
        # 확률값을 기준으로 상위 7개 결과만 추출한다
        y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
        # 금융 변수 id를 Y_ret에 저장한다
        Y_ret.append([ip for y,p,ip in y_prods])
        y_prods = [p for y,p,ip in y_prods]
        # 파일에 “고객 식별 번호, 7개의 금융 변수”를 쓴다
        f.write(("%s,%s\n" % (int(c), " ".join(y_prods))).encode('utf-8'))
    # 상위 7개 예측값을 반환한다
    return Y_ret

In [40]:
test_add_list_lightgbm = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.lightgbm.csv.gz"%str_date, "wb"), Y_test_lgbm - Y_prev, C)

NameError: name 'cv' is not defined

### 4. 캐글 업로드

### 5. 요약